In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, f1_score


In [2]:
df = pd.read_csv(r'C:\Users\acer\OneDrive - Trường ĐH CNTT - University of Information Technology\NCKH\Machine Learning_Python\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()


In [4]:
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

In [5]:
df = df.drop(columns=['customerID'])

In [6]:
df_encoded = pd.get_dummies(df, drop_first=True)

In [7]:
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# In kết quả kiểm tra
print(f"Tổng số mẫu: {len(X)}")
print(f"Số lượng Train (80%): {len(X_train)}")
print(f"Số lượng Test (20%): {len(X_test)}")

Tổng số mẫu: 7032
Số lượng Train (80%): 5625
Số lượng Test (20%): 1407


In [12]:
# 5. MÃ HÓA TOÀN BỘ CÁC CỘT CÒN LẠI (QUAN TRỌNG NHẤT)
# Lệnh này sẽ tự động tìm TẤT CẢ các cột dạng chữ (Gender, Partner, PaymentMethod...) 
# và chuyển hết thành số (One-Hot)
X_encoded = pd.get_dummies(X) 

# Kiểm tra lại xem còn cột nào là object (chữ) không
print("Kiểm tra kiểu dữ liệu (phải toàn là int/float):")
print(X_encoded.dtypes.unique())

Kiểm tra kiểu dữ liệu (phải toàn là int/float):
[dtype('int64') dtype('float64') dtype('bool')]


In [15]:
# --- BƯỚC 3: CHIA TẬP TRAIN - TEST (80% - 20%) ---
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# --- BƯỚC 4: CHUẨN HÓA DỮ LIỆU (SCALING) ---
# KNN tính khoảng cách nên bắt buộc phải Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# --- BƯỚC 5: HUẤN LUYỆN MÔ HÌNH (KNN) ---
# Chọn K=37 (Căn bậc 2 của số lượng mẫu ~ 1500, chọn số lẻ để tránh hòa)
k = 37 
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train_scaled, y_train)

print(f"\nĐã huấn luyện xong mô hình KNN với k={k}")


Đã huấn luyện xong mô hình KNN với k=37


In [17]:
acc_list = []

for k in range(1, 16):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    acc_list.append(accuracy_score(y_test, y_pred))

best_k = np.argmax(acc_list) + 1
print("Best K =", best_k)

Best K = 12


In [19]:
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train)

y_pred_best = knn_best.predict(X_test_scaled)

print("Accuracy with K= ", best_k, accuracy_score(y_test, y_pred_best))
print("Recall with K = ", best_k, ":  ", recall_score(y_test, y_pred_best))
print("F1-score with K = ", best_k, ":", f1_score(y_test, y_pred_best))

Accuracy with K=  12 0.7768301350390903
Recall with K =  12 :   0.4786096256684492
F1-score with K =  12 : 0.5327380952380952


In [20]:
y_pred_best

array([0, 0, 0, ..., 0, 0, 0], shape=(1407,))